# Introduction to Probabilistic Methods Exercises

**2025 Introduction to Quantiative Methods in Finance**

**The Erdös Institute**


These are exercises for practice, not to be submitted.

## Exercise 1 ## 

Design and code a probabilistic method to estimate $\pi\sim 3.14$.

In [1]:
import numpy as np
from math import factorial

In [40]:
# get N random points in the square [-1,1] x [-1,1] and check how many of them are in the circle centered at (0,0) with radius 1
# then comparison the areas yields the estimate of pi to be the number of points in the circle divided by N times 4 

N = 10000

square = [2*np.random.rand(2) - 1 for i in range(N)]
circle = [point for point in square if (point[0])**2+(point[1])**2 < 1]

estimated_pi = len(circle)/len(square)*4

print(f"The estimated value of pi using Monte-Carlo Method is {estimated_pi}")

The estimated value of pi using Monte-Carlo Method is 3.1684


## Exercise 2 ## 

###  Rules of Craps (Vegas Casino Style)

 
The basic rules are outlined below and a several common betting options are given.

### 1. The Come-Out Roll
 - The game begins with the shooter making a "come-out" roll using two six-sided dice.
 - If the come-out roll is a **7 or 11**, the pass line bet wins (called a "natural").
 - If the come-out roll is **2, 3, or 12**, the pass line bet loses (called "craps").
 - If the come-out roll is **4, 5, 6, 8, 9, or 10**, that number becomes the **point**.

### 2. Point Phase
 - Once the point is established, the shooter continues rolling.
 - The goal is to roll the **point number again before rolling a 7**.
     - If the shooter rolls the point again, pass line bets win.
     - If the shooter rolls a **7 before the point**, pass line bets lose (called "seven out").

### 3. Some Betting Options
 - **Pass Line Bet**: Wins on a natural (7 or 11), loses on craps (2, 3, 12), wins if point is hit before a 7.
 - **Don't Pass Bet**: Opposite of pass line. Wins on craps (2, 3), ties on 12, loses on 7 or 11.

 
 
 ### Part 1
 Determine the odds of winning at craps for the Pass Line Bet and Don't Pass Bet.
 
 ### Part 2
 Use probabilistic/Monte-Carlo methods for estimating the odds of winning the Pass Line and Don't Pass Line Bets.

In [205]:
values = range(2,13)
probabilities = np.array([1,2,3,4,5,6,5,4,3,2,1]) / 36

The odds of winning at craps for the Pass Line Bet (PLB) is computed as follows:

\begin{align}
P(\text{Winning for PLB}) & = P(\textit{first roll} = 7 \text{ or } 11) + P(roll = i  \text{ before } roll = 7 | \textit{first roll} = 4,5,6,8,9 \text{ or } 10) \\
                            & = \frac{6}{36} + \frac{2}{36} + \sum_{4 \leq i \leq 10, i \neq 7} P(\textit{first roll} = i)*P(roll = i | roll = i \text{ or } roll = 7) \\
                            & = \frac{2}{9} + \sum_{4 \leq i \leq 10, i \neq 7} P(roll = i )* \frac{P(roll=i)}{P(roll = i \text{ or } roll =7)} \\
                            & = \frac{2}{9} + \sum_{4 \leq i \leq 10, i \neq 7} \frac{P(roll=i)^2}{P(roll = i) + P(roll =7)} \\
                            & = \frac{2}{9} + \frac{2}{36}(\frac{3^2}{3+6} + \frac{4^2}{4+6} + \frac{5^2}{5+6} ) \\
                            & = 0.49\overline{29}
\end{align}




In [216]:
true_pass_line_prob = 0

for i in range(2,13):
    if i in (7,11):
        true_pass_line_prob += probabilities[i-2]
    if i in (4,5,6,8,9,10):
        true_pass_line_prob += (probabilities[i-2]**2)/(probabilities[i-2] + probabilities[5])

print(f"The probability of winning at craps for the Pass Line Bet is {true_pass_line_prob}")

The probability of winning at craps for the Pass Line Bet is 0.4929292929292929


The odds of winning at craps for the Don't Pass Line Bet (DPL) is computed as follows:

\begin{align}
P(\text{Winning for DPL}) & = P(\textit{first roll} = 2,3 \text{ or } 12) + P(roll = 7  \text{ before } roll = \textit{first roll} | \textit{first roll} = 4,5,6,8,9 \text{ or } 10) \\
                            & = \frac{2}{36} + \frac{2}{36} + \sum_{4 \leq i \leq 10, i \neq 7} P(\textit{first roll} = i)*P(roll = 7 | roll = i \text{ or } roll = 7) \\
                            & = \frac{1}{9} + \sum_{4 \leq i \leq 10, i \neq 7} P(roll = i )* \frac{P(roll=7)}{P(roll = i \text{ or } roll =7)} \\
                            & = \frac{1}{9} + \sum_{4 \leq i \leq 10, i \neq 7} \frac{P(roll=i)*P(roll=7)}{P(roll = i) + P(roll =7)} \\
                            & = \frac{1}{9} + \frac{2}{36}(\frac{3*6}{3+6} + \frac{4*6}{4+6} + \frac{5*6}{5+6} ) \\
                            & = 0.50\overline{70}
\end{align}

In [221]:
true_dpl_prob = 0

for i in range(2,13):
    if i in (2,3,12):
        true_dpl_prob += probabilities[i-2]
    if i in (4,5,6,8,9,10):
        true_dpl_prob += (probabilities[i-2]*probabilities[5])/(probabilities[i-2] + probabilities[5])

print(f"The probability of winning at craps for the Pass Line Bet is {true_dpl_prob}")

The probability of winning at craps for the Pass Line Bet is 0.507070707070707


In [206]:
def pass_line_bet():
    first_roll = np.random.choice(values, p = probabilities)
    if first_roll in (7,11):
        return True
    if first_roll in (2,3,12):
        return False
    
    point = first_roll
    current_draw = np.random.choice(values, p=probabilities)

    while current_draw not in (point,7):
        current_draw = np.random.choice(values, p=probabilities)

    if current_draw == point:
        return True
    else: return False

In [219]:
M = 100
win_probabilities = np.zeros(M)

for i in range(M):
    N = 1000
    win = 0

    for j in range(N):
        if pass_line_bet():
            win +=1

    win_probabilities[i] = win/N

print(f"The estimated probability of winning Pass Line Bet is {win_probabilities.mean():.3f}")

The estimated probability of winning Pass Line Bet is 0.491


In [114]:
def dont_pass_bet():
    first_roll = np.random.choice(values, p = probabilities)
    if first_roll in (7,11):
        return False
    if first_roll in (2,3,12):
        return True

    point = first_roll
    current_draw = np.random.choice(values, p=probabilities)

    while current_draw not in (point,7):
        current_draw = np.random.choice(values, p=probabilities)

    if current_draw == point:
        return False
    else: return True    

In [115]:
M = 100
win_probabilities = np.zeros(M)

for i in range(M):
    N = 1000
    win = 0

    for j in range(N):
        if dont_pass_bet():
            win +=1

    win_probabilities[i] = win/N

print(f"The estimated probability of winning Don't Pass Bet is {win_probabilities.mean():.3f}")

The estimated probability of winning Don't Pass Bet is 0.509


## Exercise 3 ##

Ten cards labeled 1 through 10 are shuffled. What is the probability that one of the ten cards was shuffled to its original location?

Find the true expectation and try using a probabilistic method to estimate the answer.

**Solution**

The true probability is found as follows:

For $1 \leq k \leq n$, let $S_k$ be the number of permutations of $n$ cards such that $k^{th}$ card is $k$. Then, we want to find $|S_1 \cup \dots \cup S_n|$.

\begin{align}
|S_1 \cup \dots \cup S_n| & = \sum_i |S_i| - \sum_{i<j} |S_i \cap S_j| + \sum_{i,j,k} |S_i \cap S_j \cap S_k| \dots + (-1)^{n+1} |S_1 \cap \dots S_n| \\
                     & = {n\choose 1}(n-1)! - {n\choose 2}(n-2)! + {n\choose 3}(n-3)! \dots + (-1)^{n+1}{n\choose n}(n-n)! \\
                     & = \sum_{i=1}^n (-1)^{i+1}{n\choose i}(n-i)! \\
                     & = n! \sum_{i=1}^n \frac{(-1)^{i+1}}{i!}.
\end{align}

Finally, we find the true probability that one of the $n$ cards is shuffled to its original locations to be
$$ \sum_{i=1}^n \frac{(-1)^{i+1}}{i!}. $$


In [42]:
def true_probability(n):
    sum = 0
    for i in range(1,n+1):
        sum += (-1)**(i+1)/factorial(i)
    return sum

n = 10
print(f"The true probability of one of the {n} cards is shuffled to its original location is {true_probability(n):.4f}")

The true probability of one of the 10 cards is shuffled to its original location is 0.6321


Recall

$$ e^x = \sum_{i=0}^\infty \frac{x^i}{i!}$$

Then, $$ \sum_{i=1}^n \frac{(-1)^{i+1}}{i!} \sim 1 - \frac{1}{e}$$ 

In [196]:
print(f"The true value of probability of one of the {n} cards is shuffled to its original location is {1 - np.exp(-1):.4f}")

The true value of probability of one of the 10 cards is shuffled to its original location is 0.6321


In [197]:
# Function that checks if a given permutation has a fixed value

def check_one_card_fixed(arr):
    for i in range(len(arr)):
        if arr[i] == i+1:
            return True
    return False

In [48]:
# Create N many shuffles of [1, 10] and count the number of shuffles with one card shuffled to its original location 
# using the function check_one_card_fixed above. Then the estimated probability is the number of such shuffles divided by N.

N = 100000 # number of trials

count = 0

for i in range(N):
    shuffle = np.arange(1,n+1) # initialize shuffle
    np.random.shuffle(shuffle) # shuffle the cards
    if check_one_card_fixed(shuffle): # check if one of the cards is shuffled to its original location
        count +=1

print(f"The estimated probability of one of the {n} cards is shuffled to its original location is {count/N:.4f}")

The estimated probability of one of the 10 cards is shuffled to its original location is 0.6338


## Exercise 4

You approach two slot machines. Each machine costs $\$.050$ to play and pays out $\$1$ for a win. You do not know the exact probability of winning on either machine, but you are told that **one has a 2/5 chance of winning**, and the **other has a 3/5 chance of winning**. You do not know which machine has which probability.

Your goal is to use probabilistic methods to compare two strategies over a total of 100 plays, to see which strategy is more profitable on average. A more profitable strategy is one that tends to allocate more plays to the machine with the higher payout probability.

### Strategies to Simulate

1. **Loss-Switch Strategy:**  
   Select a slot machine at random. Play on that machine until you lose, then switch to the other machine. Repeat this process for 100 plays.

2. **Exploration-Exploitation Strategy:**  
   Start by playing each machine 10 times. Estimate the win rate for each machine based on these initial plays. Then:
   - Continue playing the machine with the higher observed win rate.
   - After each play, update the estimated win rate.
   - If the other machine’s estimated win rate becomes higher, switch to it.
   - Repeat until 100 total plays have been made.

---

**Note:**  
While this is a simplified, hypothetical scenario, it mirrors real-world decision-making under uncertainty. For instance, in finance, a trader might have multiple strategies available for a given asset. The decision of which strategy to follow at a given time can be based on adaptive rules informed by past performance, similar in spirit to the strategies explored here.

In [189]:
# Probability distribution

values = np.array(['win','lose'])
probabilities = [np.array([2,3]) / 5, np.array([3,2]) / 5]

switch_machine = {0:1, 1:0}

def get_rates(counts,num_of_plays):
    n = len(counts)
    rates = [round(counts[i]/num_of_plays[i],2) for i in range(n)]

    return np.array(rates)

In [200]:
def loss_switch():
    i = 100
    win_counts = [0,0]
    number_of_plays = [0,0]
    machine = np.random.randint(2)
    while i > 0:
        play = np.random.choice(values, p = probabilities[machine])
        number_of_plays[machine] += 1
        i -= 1
        if play == 'win':
            win_counts[machine] += 1
        if play == 'lose':
            machine = switch_machine[machine]

    win_rates = get_rates(win_counts, number_of_plays)
    profit = sum(win_counts) - 0.05*100
    return profit

def exploration_exploitation():
    win_counts = [0,0]
    number_of_plays = [0,0]
    
    for machine in range(2):
        for i in range(10):
            play = np.random.choice(values, p = probabilities[machine])
            number_of_plays[machine] += 1
            if play == 'win':
                win_counts[machine] += 1

    win_rates = get_rates(win_counts, number_of_plays)

    machine = np.argmax(win_rates)

    i = 80

    while i > 0:
        play = np.random.choice(values, p=probabilities[machine])
        i -= 1
        number_of_plays[machine] += 1
        if play == 'win':
            win_counts[machine] += 1

        win_rates = get_rates(win_counts, number_of_plays)

        machine = np.argmax(win_rates)

    profit = sum(win_counts) - 0.05*100
    
    return profit  

In [192]:
# Play the game following both strategies N times and take the average profits

N = 100

profit_loss_switch = np.zeros(N)
profit_exp = np.zeros(N)

for i in range(N):
    profit_loss_switch[i] += loss_switch()
    profit_exp[i] += exploration_exploitation()

print(f"The average estimated profit of playing the game with loss-switch strategy is {profit_loss_switch.mean():.2f}")    
print(f"The average estimated profit of playing the game with exploration-exploitation strategy is {profit_exp.mean():.2f}")    

The average estimated profit of playing the game with loss-switch strategy is 46.62
The average estimated profit of playing the game with exploration-exploitation strategy is 51.74
